In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import math
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import Model, layers
from tensorflow.keras.layers import AveragePooling2D, Dense, Flatten, Conv2D, MaxPool2D
from absl import app, flags
from easydict import EasyDict
from tqdm import tqdm


import matplotlib.pyplot as plt
import foolbox as fb
from cleverhans.tf2.attacks import projected_gradient_descent, fast_gradient_method, carlini_wagner_l2


# Foolbox

In [ ]:
fmodel = fb.models.TensorFlowModel(model, bounds=(0,1))
attack = fb.attacks.L2CarliniWagnerAttack(binary_search_steps = 10,
        steps = 10000,
        stepsize = 0.03,
        confidence = 0,
        initial_const = .1,
        abort_early = False)
adv_foolbox, clipped_adversarials, success = attack(
    fmodel,
    x_to_attack,
    y_to_attack,
    epsilons=None
)

# Cleverhans

In [ ]:
clean_inputs = tf.reshape(x_to_attack, [10,28,28,1]);

adv_cleverhans = carlini_wagner_l2(model,clean_inputs,y=None,
               batch_size=10,
               clip_min=0.,
               clip_max=1.,
               binary_search_steps=10,
               max_iterations=10000,
               abort_early=False,
               confidence=0.,
               initial_const=.1,
               learning_rate=0.03)

# Eigene Implementation

In [ ]:
adv_eigene = cw_2_eigene_implementierung(model,x_to_attack, steps=2000, const=1)

#### L2 Distance für Foolbox

In [ ]:
l2_fb = [tf.norm(adv_foolbox[i]-x_to_attack[i]).numpy() for i in range(len(x_to_attack))]
l2_fb

#### L2 Distance für Cleverhans

In [ ]:
l2_ch = [tf.norm(adv_cleverhans[i].flatten()-x_to_attack[i]).numpy() for i in range(len(x_to_attack))]
l2_ch

#### L2 Distance für eigene Implementierung

In [ ]:
l2_own = [tf.norm(adv_eigene[i]-x_to_attack[i]).numpy() for i in range(len(x_to_attack))]
l2_own

In [ ]:
z = [x -.3 for x in range(10)]
y = [x +.3 for x in range(10)]
ax = plt.subplot(111)
w = 0.3
ax.bar(range(10),l2_ch,width=w, color='b', align='center',label='Cleverhans')
ax.bar(y,l2_own, width=w, color='g', align='center',label='own implementation')
ax.bar(z,l2_fb, width=w, color='r', align='center', label='foolbox')
handles, labels = ax.get_legend_handles_labels()
ax.legend(labels)
ax.set(title='CW2 Implementation Benchmark',ylabel='L2 Distance', xlabel='Sample No')
plt.show()

In [ ]:
res = fmodel(x_to_attack[:10])
adv_fb_res = fmodel(adv_foolbox)
adv_ch_res = fmodel(adv_cleverhans)
adv_eigen_res = fmodel(np.array(adv_eigene))

print(y_to_attack.numpy().tolist())
print([np.argmax(x) for x in res])
print([np.argmax(x) for x in adv_fb_res])
print([np.argmax(x) for x in adv_ch_res])
print([np.argmax(x) for x in adv_eigen_res])

# CNN-5 Definition

In [ ]:
shapes = {
    # 5x5 conv, 1 input, 6 outputs
    'conv_1': (5, 5, 1, 6),
    # 5x5 conv, 6 inputs, 16 outputs
    'conv_2': (5, 5, 6, 16),
    #5x5 conv as in paper, 16 inputs, 120 outputs
    'conv_3': (1, 1, 16, 120),
    # fully connected, 5*5*16 inputs, 120 outputs
    'dense_1': (5*5*16, 120),
    # fully connected, 120 inputs, 84 outputs
    'dense_2': (120, 84),
    # 84 inputs, 10 outputs (class prediction)
    'dense_3': (84, 10),
}
bias_shapes = {
    #output depth
    'conv_1': (6),
    'conv_2': (16),
    'dense_1': (120),
    'dense_2': (84),
    'dense_3': (10),
}

#conv2D with bias and relu activation

class CustomConvLayer(layers.Layer):

    def __init__(self, weights, mask, biases, strides, padding='SAME'):
        
        super(CustomConvLayer, self).__init__()
        self.w = weights
        self.m = mask
        self.b = biases
        self.s = strides
        self.p = padding

        
    def call(self, inputs):
        x = tf.nn.conv2d(inputs, tf.multiply(self.w, self.m), strides=[1, self.s, self.s, 1], padding=self.p,)
        x = tf.nn.bias_add(x, self.b,)
        return tf.nn.tanh(x)
        

#Average Pooling Layer
class CustomPoolLayer(layers.Layer):
    
    def __init__(self, k=2, padding='valid'):#padding='VALID'):
        super(CustomPoolLayer, self).__init__()
        self.k = k
        self.p = padding
    
    def call(self, inputs):
        return tf.nn.avg_pool2d(inputs, ksize=[1, self.k, self.k,1], strides=[1, self.k, self.k, 1], padding=self.p,)# data_format='NCHW')
    

        
class CustomConvLayer(layers.Layer):

    def __init__(self, shape, bias, strides, padding='SAME'):
        
        super(CustomConvLayer, self).__init__()
        self.w = self.add_weight(
            shape=shape,
            initializer='random_normal',
            trainable=True,
            name='w'
        )
        self.m = self.add_weight(
            shape=shape,
            initializer='ones',
            trainable=False,
            name='m'
        )
        self.b = self.add_weight(
            shape = (shape[-1]),
            initializer = 'zeros',
            trainable = True,
            name='b'
        )
        self.s = strides
        self.p = padding

        
    def call(self, inputs):
        x = tf.nn.conv2d(inputs, tf.multiply(self.w, self.m), strides=[1, self.s, self.s, 1], padding=self.p,)# data_format='NCHW')
        x = tf.nn.bias_add(x, self.b,)# 'NC...')
        return tf.nn.tanh(x)

#Dense Layer with Bias
class CustomDenseLayer(layers.Layer):
    def __init__(self, shape, bias, activation = 'tanh'):
        super(CustomDenseLayer, self).__init__()
        self.w = self.add_weight(
            shape = shape,
            initializer='random_normal',
            trainable = True,
            name='w'
        )
        self.m = self.add_weight(
            shape = shape,
            initializer='ones',
            trainable = False,
            name='m'
        )
        self.b = self.add_weight(
            shape = (shape[-1]),
            initializer = 'zeros',
            trainable = True,
            name='b'
        )
        self.a = activation
        
        
    def call(self, inputs):
        x = tf.matmul(inputs, tf.multiply(self.w, self.m))
        x = tf.nn.bias_add(x, self.b)
        if self.a == 'tanh':
            return tf.nn.tanh(x)
        if self.a == 'softmax':
            return tf.nn.softmax(x)
        if self.a == None:
            return x
        
        
        
class CustomConvModel(tf.keras.Model):
    def __init__(self):
        super(CustomConvModel, self).__init__()
        self.conv1 = CustomConvLayer(shapes['conv_1'], True, 1, 'SAME')#'VALID')
        self.maxpool1 = CustomPoolLayer(k=2, padding='SAME')
        self.conv2 = CustomConvLayer(shapes['conv_2'], True, 1, 'VALID')
        self.maxpool2 = CustomPoolLayer(k=2, padding='VALID')

        self.dense1 = CustomDenseLayer(shapes['dense_1'], True, 'tanh')
        self.dense2 = CustomDenseLayer(shapes['dense_2'], True, 'tanh')
        self.dense3 = CustomDenseLayer(shapes['dense_3'], True, None)
        self.pre_softmax = None
        
    def call(self, inputs):
        x = tf.reshape(inputs, shape=[-1,28, 28, 1])
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = layers.Flatten()(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x =  self.dense3(x)
        self.pre_softmax = x
        return tf.nn.softmax(x)
    
    

        


# Helper Functions

In [ ]:
def train_model(model, to_convergence=True):
    if to_convergence == True:
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
        model.fit(
            x=x_train,
            y=y_train,
            batch_size=64,
            epochs=500,
            callbacks=[callback],
            validation_data=(x_test, y_test),
            )
    if to_convergence == False:
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
        model.fit(
            x=x_train,
            y=y_train,
            batch_size=64,
            epochs=100,
            callbacks=[callback],
            validation_data=(x_test, y_test),
            )
    return model
def initialize_base_model(index, experiment_name, save_weights=False):

    model = CustomConvModel()

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
                  metrics=['accuracy'],
                  experimental_run_tf_function=False
                 )

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    model.fit(x=x_train,
              y=y_train,
              batch_size=64,
              epochs=1,
              callbacks=[callback],
              validation_data=(x_test, y_test),
             )
    if save_weights == True:
        model.save_weights(f'./saved-weights/{experiment_name}-{index}')
    return model


# Load Data

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

x_to_attack = tf.convert_to_tensor(x_train[:10].reshape(10,28*28))
y_to_attack = tf.convert_to_tensor([y_train[:10]])[0];

# Initialize and Train Model

In [ ]:
model = initialize_base_model(999,'')
train_model(model)

# Eigene Implementation

In [ ]:
def cw_2_eigene_implementierung(model, xs, const=1, steps=1000, learning_rate=.03, ):
    xa=[]
    
    for x in xs:
        w = tf.Variable(initial_value=tf.zeros(x.shape))
        
        show_adv = []
        optimizer = tf.keras.optimizers.Adam(.03)
        target = find_second_most_probable_class(get_logits(model,x))
        adv_image = .5 * (tf.tanh(w) + 1)
        best_adv_image = adv_image
        best_l2_dist = tf.norm(adv_image - x).numpy()
        for i in tqdm(range(steps)):
            with tf.GradientTape() as tape:
                adv_image = .5 * (tf.tanh(w) + 1)
                preds = model(adv_image)
                loss = loss_function(model= model, x=x, adv_image=adv_image, const=const, target=target)
                l2_distance = tf.norm(adv_image - x).numpy()
                if l2_distance < best_l2_dist and np.argmax(preds) == target:
                    best_adv_image = adv_image
                    best_l2_dist = l2_distance
                grads = tape.gradient(loss, w)
                optimizer.apply_gradients([(grads, w)])
        xa.append(best_adv_image)
    return xa

In [ ]:
def loss_function(model, x, adv_image, const, target):
    l1 = tf.square(tf.norm(adv_image - x))
    logit_of_best_other = get_logit_of_best_except_target(model, adv_image, target)
    logit_of_target = get_logit_of_target(model, adv_image,target)
    l2 = const * tf.math.maximum(
        logit_of_best_other
        - logit_of_target
        , 0 )
    return l1 + l2


def find_second_most_probable_class(logits):
    logits = logits.numpy().flatten()
    return np.argpartition(logits,len(logits)-2)[len(logits)-2]

        


def get_logit_of_best_except_target(model, adv_x, target):
    all_logits = get_logits(model, adv_x)
    #print('all_logits', all_logits)
    most_probable_class = tf.math.argmax(all_logits)
    second_most_probable_class = find_second_most_probable_class(all_logits)
    if target == most_probable_class: 
        return all_logits[second_most_probable_class ]
    if target != most_probable_class: 
        return all_logits[most_probable_class ]

def get_logit_of_target(model, adv_x, target):
    all_logits = get_logits(model, adv_x)
    return all_logits[target]

def get_logits(model, x):
    model(x)
    return model.pre_softmax[0]
    
